# Numerical Simulation of Electrochemical Methods

This script is adapted from "Demystifying Mathematical Modelling of Electrochemical Systems" by Stephens and Mauzeroll. The code provided in the SI of this paper is written in Matlab, and the work below adapts it into a functioning python script. 

## Linear Sweep Voltammetry 

This code specifically applies to a linear sweep voltamogram for the reduction: Fe(CN)6^3- + e^- -> Fe(CN)6^3-. Many of the other parameters have also been specified by the problem of interest. These paramerts may be altered to represent different redox systems, or even different electrochemical methods. 

In [30]:
import numpy as np 
import matplotlib.pyplot as plt
from numpy import matlib

### Simulation Input

In [31]:
# Constants
F = 96485 # Faraday’s constant [C/mol]
R = 8.314 # Gas constant [(C*V)/(mol*K)]
T = 298 # Temperature [K]

#Electrochemical parameters 
# Potential waveform
E_start = 0 # Start potential [V] {default: 0}
E_end = -0.8; # End potential [V] {default: -0.8}
E_0 = -0.4 #Standard potential [V] {default: -0.4}
nu = 10 # Scan rate [mV/s] {default: 10}


# Redox mediator
D = 1E-6 # Diffusion coefficient [cm^2/s] {default: 1E-6}
c_ox = 1 # Bulk concentration of oxidized species [M] {default: 1}
c_red = 1E-3 # Bulk concentration of reduced species [M] {default: 1E-3}
n = 1 # Number of electrons transferred during reaction {default: 1}
alpha = 0.5 # Transfer coefficient (unitless) {default: 0.5}

# Electrode
A = 1 # Electrode area [cm^2] {default: 1}
k_0 = 1E-1  # Heterogeneous rate constant [m/s] {default: 1E-1}

# Finite difference parameters 
npts_x = 100 # Number of mesh points {default: 100}
npts_t = 500 # Number of time steps {default: 500}

### Time 0 - Initial Setup and Time Step

In [32]:
# Discretize x
total_x = 0.1 # Maximum x value to simulate [cm]
del_x = total_x/npts_x # x interval
# x = np.array(range(0, total_x, del_x)) # Discretized x

# Discretize t
total_t = abs(E_end - E_start)/(nu/1000) # Maximum t value to simulate (1 LSV)
del_t = total_t/npts_t # Time interval
# t = np.array(range(0, total_t, del_t))

# Set uniform concentration along discretized x
cox_x = matlib.repmat(c_ox, npts_x,npts_t)
cred_x = matlib.repmat(c_red, npts_x,npts_t)

# Setup empty matrices for time-dependent quantities that will be filled in as the simulation progresses
E_t = np.zeros((npts_t,1))
kred_t = np.zeros((npts_t,1))
i_t = np.zeros((npts_t,1))

# Calculate initial potential
E_curr = E_start # E_curr = Current potential (single value)
E_t[0] = E_curr # E_t = Potential waveform (matrix of values)


# Calculate formal potential according to the Nernst equation
E_eq = E_0 - ((R*T)/(n*F) * np.log(c_ox/c_red))

# Calculate initial rate constant according to the Butler-Volmer equation
eta = E_curr - E_eq # Overpotential (V)
kred_curr = k_0*np.exp((-alpha*n*F*eta)/(R*T))

# # Concentration in box 0 updated due to chemical reaction
# cred_x[0,0] = (cred_x[0] + (del_t*(kred_curr * cox_x[0,0])))[0] # Equation 17
# cox_x[0,0] = cox_x[0] - (del_t*(kred_curr * cox_x[0,0])) # Equation 20

# Current is calculated based on change in concentration
# i_t[0] = -n*F*(del_x*(cred_x[0,0] - c_red) / del_t)
i_t[0] = -0.00012644 

# cred_x[0,0] = c_red

### Time > 0: Simulation Continues

In [42]:
cred_temp = cred_x
cox_temp= cox_x
iter = np.ones(1)

for j in range(1, npts_t): #Iterate over number of time points
    #Pull concentration profiles from end of previous time step
    cred_x[:,j] = cred_x[:,j-1]
    cox_x[:,j] = cox_x[:,j-1]
    iter[0] = j

    # #Update concentration due to diffusion: Equation 26 
    # for i in range(1, npts_x): # Iterate over number of boxes
    # #     # Note: a temporary variable is used here to make sure that the
    # #     # concentration for this time step does not change as it is being
    # #     # used in the calculation
    #     if i == 0:
    #         cred_temp[i,j] = cred_x[i,j]  + (D*del_t/(del_x**2)) * ((cred_x[i+1,j] - cred_x[i,j]))
    #         cox_temp[i,j] = cox_x[i,j]  + (D*del_t/(del_x**2)) * ((cox_x[i+1,j] - cox_x[i,j]))

    #     # In the last box (i=max), we apply our boundary condition of
    #     # concentration = bulk concentration
    #     elif i == npts_x-1:
    #         cred_temp[i,j] = c_red
    #         cox_temp[i,j] = c_ox

    #     # #In all other boxes, we use the diffusion expression (Equation 26) as normal
    #     else:
    #         cred_temp[i,j] = cred_x[i,j] + (D*del_t/(del_x**2))*((cred_x[i+1,j] - 2*cred_x[i,j] + cred_x[i-1,j]))
    #         cox_temp[i,j] = cox_x[i,j] + (D*del_t/(del_x**2))*((cox_x[i+1,j] - 2*cox_x[i,j] + cox_x[i-1,j]))


    cred_x[:,j] = cred_temp[:,j]
    cox_x[:,j] = cox_temp[:,j]

    #Update potential: Equation 12 %%%
    E_curr = E_curr - del_t*((nu)/1000)
    E_t[j] = E_curr

    # Update rate constant: Equation 14 
    eta = E_curr - E_eq # Overpotential (V)
    kred_curr = k_0* np.exp((-alpha*n*F*eta)/(R*T))
    kred_t[j] = kred_curr


    # # Chemical reaction in box 0: Equations 17 & 20 
    cred_x[0,j] = cred_x[0,j-1] + (del_t*(kred_curr * cox_x[0,j-1]))
    cox_x[0,j] = cox_x[0,j-1] - (del_t*(kred_curr * cox_x[0,j-1]))

    # Calculate current: Equation 22 
    i_t[j] = -n*F*((del_x*(cred_x[0,j] - cred_x[0,j-1])) / del_t)

C:\Users\IBIS\AppData\Local\Temp\ipykernel_5748\433931162.py:41: RuntimeWarning: overflow encountered in exp
  kred_curr = k_0* np.exp((-alpha*n*F*eta)/(R*T))


OverflowError: cannot convert float infinity to integer

In [ ]:
# fig = plt.figure(figsize = (7,5))
# plt.plot(E_t,i_t)